![Two data scientists working on a dashboard.](images/hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [18]:
import pandas as pd
import numpy as np

In [19]:
ds_jobs = pd.read_csv('data/customer_train.csv')
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [20]:
ds_jobs.head(10)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0
5,21651,city_176,0.764,NaN,Has relevant experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1
6,28806,city_160,0.920,Male,Has relevant experience,no_enrollment,High School,NaN,5,50-99,Funded Startup,1,24,0
7,402,city_46,0.762,Male,Has relevant experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1
8,27107,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1
9,699,city_103,0.920,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0


In [51]:
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [53]:
print(f'The total memory usage of this dataset is {ds_jobs.memory_usage().sum():,} bytes')

The total memory usage of this dataset is 2,145,824 bytes


In [54]:
ds_jobs.isna().sum()

student_id                   0
city                         0
city_development_index       0
gender                    4508
relevant_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
job_change                   0
dtype: int64

For the purposes of this mini-project, we will not clean up the dataset, since dropping rows will result in a more than 50% of the data. Needs more work to infer the missing values.

In [56]:
ds_jobs_clean = ds_jobs.copy()

First step: convert features to corresponding data types to optimize memory usage.

In [58]:
ds_jobs_clean['student_id']=ds_jobs_clean['student_id'].astype(np.int32)
ds_jobs_clean['training_hours']=ds_jobs_clean['training_hours'].astype(np.int32)
ds_jobs_clean['job_change']=ds_jobs_clean['job_change'].astype(np.int32)
ds_jobs_clean['city_development_index']=ds_jobs_clean['city_development_index'].astype(np.float16)
ds_jobs_clean['gender']=ds_jobs_clean['gender'].astype('category')
ds_jobs_clean['relevant_experience']=ds_jobs_clean['relevant_experience'].astype('category')
ds_jobs_clean['enrolled_university']=ds_jobs_clean['enrolled_university'].astype('category')
ds_jobs_clean['education_level']=ds_jobs_clean['education_level'].astype('category')
ds_jobs_clean['major_discipline']=ds_jobs_clean['major_discipline'].astype('category')
ds_jobs_clean['experience']=ds_jobs_clean['experience'].astype('category')
ds_jobs_clean['company_size']=ds_jobs_clean['company_size'].astype('category')
ds_jobs_clean['company_type']=ds_jobs_clean['company_type'].astype('category')
ds_jobs_clean['last_new_job']=ds_jobs_clean['last_new_job'].astype('category')
ds_jobs_clean['city']=ds_jobs_clean['city'].astype('category')
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  int32   
dtypes: category(10), float


Step 2: Categorize 'education_level', 'experience', 'company_size', 'last_new_job', 'relevant_experience' and 'enrolled_university' features with ordered categories to reflect the natural order of the columns.

In [59]:
ds_jobs_clean['education_level'].value_counts()

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64

In [31]:
ds_jobs_clean['education_level'] = ds_jobs_clean['education_level'].cat.set_categories(['Primary School','High School','Graduate','Masters','Phd'], ordered=True)

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64

In [32]:
ds_jobs_clean['experience'].value_counts()

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: experience, dtype: int64

In [36]:
ordered_experience =['<1','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','>20']
ds_jobs_clean['experience'].cat.set_categories(ordered_experience, ordered=True, inplace=True)

/var/folders/p7/cshrl10s47n6p2hd0yjwjjsr0000gn/T/ipykernel_94526/4177344659.py:2: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  ds_jobs_clean['experience'].cat.set_categories(ordered_experience, ordered=True, inplace=True)


In [37]:
ds_jobs_clean['company_size'].value_counts()

50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64

In [38]:
ordered_company_size=['<10','10-49','50-99','100-499','500-999','1000-4999','5000-9999','10000+']
ds_jobs_clean['company_size'].cat.set_categories(ordered_company_size,ordered=True, inplace=True)

/var/folders/p7/cshrl10s47n6p2hd0yjwjjsr0000gn/T/ipykernel_94526/4013541145.py:2: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  ds_jobs_clean['company_size'].cat.set_categories(ordered_company_size,ordered=True, inplace=True)


In [39]:
ds_jobs_clean['last_new_job'].value_counts()

1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
Name: last_new_job, dtype: int64

In [40]:
ordered_last_job = ['never','1','2','3','4','>4']
ds_jobs_clean['last_new_job'].cat.set_categories(ordered_last_job,ordered=True, inplace=True)

/var/folders/p7/cshrl10s47n6p2hd0yjwjjsr0000gn/T/ipykernel_94526/3621669002.py:2: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  ds_jobs_clean['last_new_job'].cat.set_categories(ordered_last_job,ordered=True, inplace=True)


In [41]:
ds_jobs_clean['relevant_experience'].value_counts()

Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64

In [42]:
ds_jobs_clean['relevant_experience'].cat.set_categories(['No relevant experience','Has relevant experience'], ordered=True, inplace=True)

/var/folders/p7/cshrl10s47n6p2hd0yjwjjsr0000gn/T/ipykernel_94526/1944120658.py:1: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  ds_jobs_clean['relevant_experience'].cat.set_categories(['No relevant experience','Has relevant experience'], ordered=True, inplace=True)


In [43]:
ds_jobs_clean['enrolled_university'].value_counts()

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64

In [44]:
ds_jobs_clean['enrolled_university'].cat.set_categories(['no_enrollment','Part time course','Full time course'], ordered=True, inplace=True)

/var/folders/p7/cshrl10s47n6p2hd0yjwjjsr0000gn/T/ipykernel_94526/2950786591.py:1: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  ds_jobs_clean['enrolled_university'].cat.set_categories(['no_enrollment','Part time course','Full time course'], ordered=True, inplace=True)


If we compare ds_jobs and ds_jobs_clean, we can see that the memory usage of ds_jobs_cless is much smaller.

In [60]:
print(f'The total memory usage of data_jobs dataset is {ds_jobs.memory_usage().sum():,} bytes')
print(f'The total memory usage of data_jobs_clean dataset is {ds_jobs_clean.memory_usage().sum():,} bytes')

The total memory usage of data_jobs dataset is 2,145,824 bytes
The total memory usage of data_jobs_clean dataset is 467,428 bytes


So the porcentage of the memory usage reduction is approximately 78%

In [45]:
filter_conditions=(ds_jobs_clean['experience']>='10')&(ds_jobs_clean['company_size']>='1000-4999')
ds_jobs_clean = ds_jobs_clean[filter_conditions]

In [47]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   int32   
dtypes: category(10), float1

In [50]:
ds_jobs_clean.memory_usage().sum()

77940